This entire notebook is just a sketch of how the recursive algorithm ought to behave. It uses the ISL python bindings and polylib to construct the face lattice.

# Table of Contents

* Helper functions
* Actual algorithm
* Manual example

# Helper functions

In [1]:
from face_lattice import *   # this is my implementation of Loechner & Wilde's ScanFaces algorithm
from islpy import *
from functools import reduce
from copy import deepcopy

#### `inverse(op)`
Given an operation `op`, return its inverse. This is just hardcoded with '+' and 'max' for now.

In [2]:
def inverse(op):
    store = {
        '+': '-',
        'max': None,
        # ...
    }
    if op not in store:
        raise Exception('Operator "{}" not supported yet.'.format(op))
    return store[op]

#### `ker_from_map(f)`
Given an ISL map `f`, return the ISL set of points in the null space of `f`

In [3]:
def ker_from_map(f):
    if type(f) == str:
        f = BasicMap(f)
    mat = []
    for c in f.get_constraints():
        vars = c.get_var_dict()
        index_vals = [-1 * int(c.get_coefficient_val(v[0], v[1]).to_str()) for k, v in vars.items() if
                      v[0] != dim_type.param]
        mat.append(index_vals)
    mat = np.array(mat)
    
    indices = ','.join(['i{}'.format(i) for i in range(len(mat[0, :]))])
    constraints = []
    for c in mat:
        constraints.append(
            '+'.join(['{}{}'.format(a[0], a[1]) for a in zip(c, indices.split(','))]) + '=0')
    s = '{{[{}] : {}}}'.format(indices, ' and '.join(constraints))
    return BasicSet(s)

# examples
print('ex1: {}'.format(ker_from_map('{[i,j,k]->[i]}')))
print('ex2: {}'.format(ker_from_map('{[i,j,k]->[i,k]}')))
print('ex3: {}'.format(ker_from_map('{[i,j,k]->[i+k]}')))

ex1: { [i0, i1, i2] : i0 = 0 }
ex2: { [i0, i1, i2] : i0 = 0 and i2 = 0 }
ex3: { [i0, i1, i2] : i2 = -i0 }


#### `ker_from_facet_normal(facet, C)`
Given a facet `facet` (i.e., set of rows indices from `C` that are saturated), return the ISL set of points in the null space of the vector normal to the facet. 

For example, consider the $i=0$ face from the following set (3D cube),

$$ \{[i,j,k] : 0 \le i,j,k \le 100\} $$

The normal vector to the $i=0$ face is $[1,0,0]$ (and is equivalent to the vector representation of this constraint), therefore the null space is given by the set,

$$ \{[i,j,k] : i=0 \} $$

These are the points "in the facet".

In [4]:
def ker_from_facet_normal(facet, parent, C):
    np_C = np.array(C)
    mat = np_C[np.array(list(facet-parent)),:-1]
    indices = ','.join(['i{}'.format(i) for i in range(len(mat[0, :]))])
    constraints = []
    for c in mat:
        constraints.append('+'.join(['{}{}'.format(a[0], a[1]) for a in zip(c, indices.split(','))]) + '=0')
    s = '{{[{}] : {}}}'.format(indices, ' and '.join(constraints))
    return BasicSet(s)

#### `is_strict(facet, fp, C)`
Given a facet, represented by `facet` & `C`, and the projection function `fp`, determine whether the facet is considered a strict boundary. We'll call a facet a strict boundary if both of the following hold,

$$ ker(f_{p}) \cap ker(c) \ne \emptyset $$
$$ ker(f_{p}) \supseteq ker(c) $$

In [5]:
def is_strict(facet, parent, fp, C):
    ker_c = ker_from_facet_normal(facet, parent, C)
    ker_fp = ker_from_map(fp)
    return not ker_c.intersect(ker_fp).is_empty() and ker_fp.is_subset(ker_c)

#### `rho_from_labels(facets, C, labels, fd)`

Given a labeling (i.e., list of labels `labels`) and dependence function `fd`, return a $\rho$ that induces the labeling if possible. 
Each `facet` in `facets` is a set of indices of `C` that when saturated, describe the facet.
Each `label` in `labels` is either the string 'ADD', 'INV', or 'SUB' (if operator admits an inverse).

For example, if `facets = [{0,1},{0,2},{0,4}]` and `labels = ['ADD','ADD','INV']`, then the set that satifies the following constraints constitues the feasible space of valid $\rho$,

$$ \rho \cdot c_{0} > 0 $$
$$ \rho \cdot c_{1} > 0 $$
$$ \rho \cdot c_{2} = 0 $$

where $c_{i}$ denotes the normal vector of the $i$'th facet. Here $i=0$ corresponds to `facets[0]`, or constraints 0 and 1 from `C`. And $i=1$ denotes `facets[1]` or constraints 0 and 2. The conditions '>', '=', or '<' are chosen based on the labels 'ADD', 'INV', and 'SUB' respectively. Additionally, $\rho$ must be in the null space of the dependence function `fd`.

If this set is not trivially empty then we conclude that the labeling is valid. Any $\rho$ in the set is a valid $\rho$ that induces these labels, and we can just select the first choice (via lexmin or lexmax). Otherwise it returns None.

In [6]:
def rho_from_labels(faces, parent, Lp, C, labels, fd):
    # check if this combo is possible given the feasible_rho
    # c1     c2     c3
    # 'ADD', 'ADD', 'INV'
    # c1*rho>=0 and c2*rho>=0 and c3*rho=0  and in ker(fp) and it saturates current node
    # {[i,j,k] : j>=0 and i-j-k>=0 and k=0  and i=0   }  <- is this empty?  if yes, then prune out this combo
    # if no, then check that it's "in ker(fp)
    map = {'ADD': '>', 'INV': '=', 'SUB': '<'}
    bsets = list()
    bsets.append(Lp)
    # must be in ker(fd)
    bsets.append(ker_from_map(fd))

    for face, label in zip(faces, labels):
        # select constraints representing this face, and create the set according to its label
        mat = C[np.array(list(face-parent)),:-1]
        indices = ','.join(['i{}'.format(i) for i in range(len(mat[0,:]))])
        constraints = []
        for c in mat:
            constraints.append('+'.join(['{}{}'.format(a[0],a[1]) for a in zip(c, indices.split(','))]) + '{}0'.format(map[label]))
        s = '{{[{}] : {}}}'.format(indices, ' and '.join(constraints))
        bsets.append(BasicSet(s))

    # check that it's not trivially just the origin
    origin = BasicSet('{{[{}]}}'.format(('0,'*(C.shape[1]-1))[:-1]))
    result = reduce(lambda s0,s1: s0.intersect(s1), bsets)

    feasible_rho = result - origin

    if not feasible_rho:
        return None

    rho = list()
    # either lexmin or lexmax, not sure yet how to determine upfront which one to use, so for now just try both
    funcs = [feasible_rho.lexmin, feasible_rho.lexmax]
    for func in funcs:
        try:
            func().foreach_point(rho.append)
        except Exception:
            continue
        finally:
            if rho:
                break

    return rho[0]

#### `enumerate_labels(facets, labels)`
At each node in the face lattice, there are one or more facets that we potentially need to recurse into. This function just enumerates all possible labelings given the set of facets and the set of possible labels. If we have 4 facets and 3 possible labels, then there are $3^{4}$ combinations of labels.

In [7]:
# convert base-10 number n to base-d
def d_ary(d, n):
    if n == 0:
        return '0'
    nums = []
    while n:
        n, r = divmod(n, d)
        nums.append(str(r))
    return ''.join(reversed(nums))

def enumerate_labels(facets, labels):
    LABEL = {i:labels[i] for i in range(len(labels))}
    num_labels = len(labels)
    num_facets = len(facets)
    for i in range(num_labels ** num_facets):
        labels = [LABEL[int(d_ary(num_labels, i).zfill(num_facets)[j])] for j in range(num_facets)]
        yield labels

# example, all possible combos of 3 facets with 2 labels (8 combos total)
for labeling in enumerate_labels([{0},{1},{2}] , ['ADD','INV']):
    print(labeling)

['ADD', 'ADD', 'ADD']
['ADD', 'ADD', 'INV']
['ADD', 'INV', 'ADD']
['ADD', 'INV', 'INV']
['INV', 'ADD', 'ADD']
['INV', 'ADD', 'INV']
['INV', 'INV', 'ADD']
['INV', 'INV', 'INV']


#### `prune_combos(label_combos)`

At each node in the face lattice, we have a list of valid facet labels.  Some of these may be redundant.  Remove redundant valid combinations. Prefer configurations that require fewer overall recursive calls.

In [8]:
def prune_combos(label_combos):
    combos = [[1 if l == 'ADD' else 0 for l in c[:-1]] for c in label_combos]
    combos.sort(key=lambda c: np.sum(c), reverse=1)
    # sort by combos with most ADD faces first
    # discard a combo if its ADD faces can
    ret = []
    for i,combo in enumerate(combos):
        _combo = np.array(combo)
        for other in combos[i+1:]:
            other = np.array(other)
            _combo = _combo - other
        if 1 in _combo:
            ret.append(combo)
    # TODO - return only label_combos rows that match ret
    unique_combos = []
    for lc in label_combos:
        labels = [1 if l == 'ADD' else 0 for l in lc[:-1]]
        if labels in ret:
            unique_combos.append(lc)
    return unique_combos

prune_combos([['INV','ADD','ADD',[2,1,0]],
              ['INV','ADD','INV',[1,1,0]],
              ['INV','INV','ADD',[1,0,0]]])

[['INV', 'ADD', 'INV', [1, 1, 0]], ['INV', 'INV', 'ADD', [1, 0, 0]]]

#### `point_to_vec(isl_point)`

Take an `islpy._isl.Point` object and return it as a python list.

In [9]:
# take an isl_point obj and parse it as a vector
# given {Point}{ [1, 1, 0] }
# return [1, 1, 0]
def point_to_vec(isl_point):
    N = len(isl_point.get_var_dict())
    vd = isl_point.get_var_dict()
    vec = [int(isl_point.get_coordinate_val(vd[v][0], vd[v][1]).to_str()) for v in vd]
    return vec

#### `boundary_label(facet, parent, C, rho)`

Given a chosen reuse vector (rho), determine whether or not a boundary facet is considered "inward" (i.e., initialization) or "outward". This is given by the dot product between rho and the facet normal vector.

In [10]:
def boundary_label(facet, parent, C, rho):
    c_mat = C[np.array(list(facet-parent)), :-1]
    rho_vec = point_to_vec(rho)
    orientations = np.matmul(c_mat, rho_vec)
    zero_vec = np.zeros(len(c_mat))
    return 'inward' if orientations >= zero_vec else 'outward'

#### `get_Lp(node, C)`

Get the effective linear space (Lp) of the node in the lattice.  This is theorem 2 from the 2006 paper, and is given by the intersection of the null spaces of each saturated constraint.

In [11]:
def get_Lp(node, C):
    # start with universe
    Lp = BasicSet('{{[{}]}}'.format(','.join(['i{}'.format(i) for i in range(C.shape[1] - 1)])))
    np_C = np.array(C)
    for c in list(node):
        mat = np_C[[c], :-1]
        indices = ','.join(['i{}'.format(i) for i in range(len(mat[0, :]))])
        constraints = []
        for c in mat:
            constraints.append('+'.join(['{}{}'.format(a[0], a[1]) for a in zip(c, indices.split(','))]) + '=0')
        s = '{{[{}] : {}}}'.format(indices, ' and '.join(constraints))
        Lp = Lp.intersect(BasicSet(s))
    return Lp

# Recursive Algorithm

Initial setup given an input reduction expression $R$ with body $E$,

* Compute the context domain `s` of the reduction body
* Construct the face lattice `lattice` from `s`
* Initialize `fp` to the projection function of $R$
* Let `k` denote the remaining number of dimensions of reuse available, initialize to the rank of the null space of the depdence function `fd` present in the expression $E$ of the reduction body (corrected for "equalities")
* Initialize `node` to the root node in the face lattice
* Call `simplify(k, fp, fd, node, lattice, C, legal_labels)`, (see below for descriptions of `C` and `legal_labels`)

Define `simplify(k, fp, fd, node, lattice, C, legal_labels` where,
* `k`, `fp`, `node`, and `lattice` are initialized as described above
* `C` is the constraints matrix of the set `s`
* `legal_labels` is either ['ADD','INV','SUB'] or ['ADD','INV'] if the reduction operation does or does not admit an inverse, respectively

1. if `k == 0`, then return SUCCESS
1. For each facet of the current node in the lattice, label as either a boundary or not `fp`
1. Construct the list of candidate facets that potentially need to be recursed into, these are the non-boundary facets
1. Enumerate all possible labelings and for each, construct the feasible space of legal reuse vectors $\rho$ (see `rho_from_labels` above). If the space for a particular labeling is empty, then conclude that the labeling is impossible. If all labelings impossible then return FAILURE.
1. Prune out redundant labelings.
1. For all possible labelings, recurse into each 'ADD' facet with $k-1$ (to indicate that there is one less dimension of reuse avilable. For a given labeling, if the recursion into all facets is successful then save the labeling as successful. As long as we check all possible labelings at a given label, then we don't need to backtrack.
1. If at least one labeling was successful then return SUCCESS and the set of all successful labelings else return FAILURE

TODO - figure out how to navigate the infinite space of reduction decompositions.
Try all possibilities (i.e., decomposition + no-decomposition) at each node, OR only do decomposition if no-decomposition fails.  But then think about how to incorporate all possible combinations.  Because it's possible that decomposition at one level will cause something else to fail at a lower level and/or vice-versa.  How to avoid backtracking.

In [12]:
def simplify(k=None, fp_str=None, fd_str=None, node=None, lattice=None, C=None, legal_labels=None, rho=None):
    def pprint(*args, **kwargs):
        print('@{} '.format(set(node) if node else '{}'), end='')
        print(*args, **kwargs)

    pprint('STEP A.1 - if k==0 return else continue. k={}'.format(k))
    if k == 0:
        pprint()
        pprint('Success - reached bottom, no more available dimensions of reuse.')
        pprint()
        return True
    pprint()

    fp = BasicMap(fp_str)
    fd = BasicMap(fd_str)
    pprint('node = {}'.format(set(node)))
    pprint('fp = {}'.format(fp_str))
    pprint('fd = {}'.format(fd_str))
    pprint()

    pprint('STEP A.2 - identify "strict" boundaries given fp')
    pprint()
    facets = list(lattice.graph.neighbors(node))
    for facet in facets:
        label = 'boundary' if is_strict(facet, node, fp, C) else ''
        pprint(set(facet), label)
    pprint()

    pprint('STEP A.3 - construct list of candidate facets (i.e., non-boundary facets)')
    pprint()
    candidate_facets = [facet for facet in facets if not is_strict(facet, node, fp, C)]
    pprint('candidate_facets = {}'.format([set(cf) for cf in candidate_facets]))
    pprint()

    pprint('STEP A.4 - determine all possible combos')
    pprint()
    label_combos = list()
    header = ['{}'.format(set(f)) for f in candidate_facets]
    pprint(header)
    pprint('-' * len(str(header)))
    # theorem 2
    Lp = get_Lp(node, C)
    for labels in enumerate_labels(candidate_facets, legal_labels):
        rho = rho_from_labels(candidate_facets, node, Lp, C, labels, fd)
        if rho:
            label_combos.append(labels + [rho])
            pprint('{}  possible -> rho = {}'.format(labels, rho))
        else:
            pprint('{}  impossible'.format(labels))
    pprint()
    if not label_combos:
        pprint('FAILURE - no possible combos')
        return False

    pprint('STEP A.5 - prune out redundant possible combos')
    pprint()
    pprint(header)
    pprint('-' * len(str(header)))
    unique_label_combos = prune_combos(label_combos)
    for combo in unique_label_combos:
        labels,rho = combo[:-1],combo[-1]
        pprint('{}  -> rho = {}'.format(labels, rho))
    pprint()

    pprint('STEP A.6 - incorporate boundary facets')
    pprint()
    boundary_facets = [f for f in facets if f not in candidate_facets]
    header = '{} {}'.format(header, [str(set(bf)) for bf in boundary_facets])
    pprint(header)
    pprint('-' * len(str(header)))
    full_label_combos = []
    for combo in unique_label_combos:
        labels,rho = combo[:-1],combo[-1]
        boundary_labels = []
        full_label_combo = deepcopy(labels)
        for boundary_facet in boundary_facets:
            bl = boundary_label(boundary_facet, node, C, rho)
            boundary_labels.append(bl)
            full_label_combo.append(bl)
        pprint('{} {}  -> rho = {}'.format(labels, boundary_labels, rho))
        full_label_combo.append(rho)
        full_label_combos.append(full_label_combo)
    pprint()

    pprint('STEP A.7 - recurse into "ADD" and "inward" boundary facets')
    pprint()
    successful_combos = []
    for combo in full_label_combos:
        labels,rho = combo[:-1],combo[-1]
        abort = None
        for label,facet in zip(labels,candidate_facets + boundary_facets):
            if label != 'ADD' and label != 'inward':
                continue
            pprint('recursing into {} facet'.format(set(facet)))
            pprint()
            ret = simplify(k=k-1, fp_str=fp_str, fd_str=fd_str, node=facet, lattice=lattice, C=C, legal_labels=legal_labels)
            abort = not ret
            if abort:
                break
            # TODO - figure out how to propagate the success back up the recusion
            # TODO - likely, attach to the face lattice accordingly
        if not abort:
            successful_combos.append(combo)
        pprint()
    if len(successful_combos) == 0:
        pprint('FAILURE - no successful combos')
    pprint()

    pprint('STEP B - todo...')
    pprint()
    # TODO - implement step 6b from Algorithm 2 in 2006 paper

    pprint('STEP C - reduction decomposition - todo...')
    pprint()
    # TODO - figure out how to do the decomposition
    # TODO - i.e., how to navigate the infinite space of decompositions

    return successful_combos

# Example 1

Take the following AlphaZ program (based on the first example from the slides):

```
affine SR {N|}
input
    float X {i | 1<=i<=100};
output
    float Y {i | 1<=i<=100};
let
    Y = reduce(+, (i,j,k->i), {| 1<=j<=i-1 && 1<=k<=i-j } : (i,j,k->k)@X);
.
```

It has the following operator (op), projection function ($f_{p}$), body context domain (s) given be AlphaZ, and dependence function ($f_{d}$):

In [13]:
op = 'max'
fp = '{[i,j,k]->[i]}'
s = '{[i,j,k] : j>=1 and i>=j+1 and k>=1 and i>=j+k and 0>=k-100 and i>=2 and 0>=i-100}'
fd = '{[i,j,k]->[k]}'

### Step 1 - construct the face lattice


In [14]:
C, lattice, bset, dim_P = face_lattice(s)

{[i,j,k] : j>=1 and i>=j+1 and k>=1 and i>=j+k and 0>=k-100 and i>=2 and 0>=i-100}

C_hat (constraints):
[[  0   0   1  -1]		{ [i, j, k] : -1 + k >= 0 }
 [ -1   0   0 100]		{ [i, j, k] : 100 - i >= 0 }
 [  0   1   0  -1]		{ [i, j, k] : -1 + j >= 0 }
 [  1  -1  -1   0]		{ [i, j, k] : i - j - k >= 0 }
 [  0   0   0   1]]		

R_hat (rays/vertices):
[[100 100   2 100]
 [  1  99   1   1]
 [  1   1   1  99]
 [  1   1   1   1]]

Dimension:
3

3-faces: [{}]
2-faces: [{0}, {1}, {2}, {3}]
1-faces: [{0, 1}, {0, 2}, {0, 3}, {1, 2}, {1, 3}, {2, 3}]
0-faces: [{0, 1, 2}, {0, 1, 3}, {0, 2, 3}, {1, 2, 3}]



### Step 2
Determine the set of legal labels based on whether or not the operator admits an inverse

In [15]:
legal_labels = ['ADD', 'INV']
if inverse(op):
    legal_labels.append('SUB')
       
print('op = {}'.format(op))
print(legal_labels)

op = max
['ADD', 'INV']


### Step 3
Get the root node from the lattice

In [16]:
root = lattice.get_root()
print(set(root))

set()


### Step 4
Start the recursion at the root node

In [17]:
simplify(k=2, fp_str=fp, fd_str=fd, node=root, lattice=lattice, C=C, legal_labels=legal_labels)

@{} STEP A.1 - if k==0 return else continue. k=2
@{} 
@{} node = set()
@{} fp = {[i,j,k]->[i]}
@{} fd = {[i,j,k]->[k]}
@{} 
@{} STEP A.2 - identify "strict" boundaries given fp
@{} 
@{} {0} 
@{} {1} boundary
@{} {2} 
@{} {3} 
@{} 
@{} STEP A.3 - construct list of candidate facets (i.e., non-boundary facets)
@{} 
@{} candidate_facets = [{0}, {2}, {3}]
@{} 
@{} STEP A.4 - determine all possible combos
@{} 
@{} ['{0}', '{2}', '{3}']
@{} ---------------------
@{} ['ADD', 'ADD', 'ADD']  impossible
@{} ['ADD', 'ADD', 'INV']  impossible
@{} ['ADD', 'INV', 'ADD']  impossible
@{} ['ADD', 'INV', 'INV']  impossible
@{} ['INV', 'ADD', 'ADD']  possible -> rho = { [2, 1, 0] }
@{} ['INV', 'ADD', 'INV']  possible -> rho = { [1, 1, 0] }
@{} ['INV', 'INV', 'ADD']  possible -> rho = { [1, 0, 0] }
@{} ['INV', 'INV', 'INV']  impossible
@{} 
@{} STEP A.5 - prune out redundant possible combos
@{} 
@{} ['{0}', '{2}', '{3}']
@{} ---------------------
@{} ['INV', 'ADD', 'INV']  -> rho = { [1, 1, 0] }
@{} ['INV'

[['INV', 'ADD', 'INV', 'outward', Point("{ [1, 1, 0] }")],
 ['INV', 'INV', 'ADD', 'outward', Point("{ [1, 0, 0] }")]]

# Example 2

Take the following AlphaZ program (based on Figure 2 from the 2006 paper & the second example from the slides):

```
affine SR {N|}
input
	float X {i,j | 1<=i,j<=100};
output
	float Y {i | 1<=i<=100};
let
	Y[i] = reduce(+, (i,j,k->i), {| 1<=j<=i && 1<=k<=100-i } : (i,j,k->j,k)@X);
.
```

It has the following operator (op), projection function ($f_{p}$), body context domain (s) given be AlphaZ, and dependence function ($f_{d}$):

In [18]:
op = 'max'
fp = '{[i,j,k]->[i]}'
s = '{[i,j,k] : j>=1 and i>=j and k>=1 and 0>=i+k-100 and 0>=k-100 and 0>=j-100 and 0>=i-99 and i>=1}'
fd = '{[i,j,k]->[j,k]}'

### Step 1 - construct the face lattice


In [19]:
C, lattice, bset, dim_P = face_lattice(s)

{[i,j,k] : j>=1 and i>=j and k>=1 and 0>=i+k-100 and 0>=k-100 and 0>=j-100 and 0>=i-99 and i>=1}

C_hat (constraints):
[[  1  -1   0   0]		{ [i, j, k] : i - j >= 0 }
 [  0   0   1  -1]		{ [i, j, k] : -1 + k >= 0 }
 [  0   1   0  -1]		{ [i, j, k] : -1 + j >= 0 }
 [ -1   0  -1 100]		{ [i, j, k] : 100 - i - k >= 0 }
 [  0   0   0   1]]		

R_hat (rays/vertices):
[[ 1 99  1 99]
 [ 1 99  1  1]
 [ 1  1 99  1]
 [ 1  1  1  1]]

Dimension:
3

3-faces: [{}]
2-faces: [{0}, {1}, {2}, {3}]
1-faces: [{0, 1}, {0, 2}, {0, 3}, {1, 2}, {1, 3}, {2, 3}]
0-faces: [{0, 1, 2}, {0, 1, 3}, {0, 2, 3}, {1, 2, 3}]



### Step 2
Determine the set of legal labels based on whether or not the operator admits an inverse

In [20]:
legal_labels = ['ADD', 'INV']
if inverse(op):
    legal_labels.append('SUB')
       
print('op = {}'.format(op))
print(legal_labels)

op = max
['ADD', 'INV']


### Step 3
Get the root node from the lattice

In [21]:
root = lattice.get_root()
print(set(root))

set()


### Step 4
Start the recursion at the root node

In [22]:
simplify(k=1, fp_str=fp, fd_str=fd, node=root, lattice=lattice, C=C, legal_labels=legal_labels)

@{} STEP A.1 - if k==0 return else continue. k=1
@{} 
@{} node = set()
@{} fp = {[i,j,k]->[i]}
@{} fd = {[i,j,k]->[j,k]}
@{} 
@{} STEP A.2 - identify "strict" boundaries given fp
@{} 
@{} {0} 
@{} {1} 
@{} {2} 
@{} {3} 
@{} 
@{} STEP A.3 - construct list of candidate facets (i.e., non-boundary facets)
@{} 
@{} candidate_facets = [{0}, {1}, {2}, {3}]
@{} 
@{} STEP A.4 - determine all possible combos
@{} 
@{} ['{0}', '{1}', '{2}', '{3}']
@{} ----------------------------
@{} ['ADD', 'ADD', 'ADD', 'ADD']  impossible
@{} ['ADD', 'ADD', 'ADD', 'INV']  impossible
@{} ['ADD', 'ADD', 'INV', 'ADD']  impossible
@{} ['ADD', 'ADD', 'INV', 'INV']  impossible
@{} ['ADD', 'INV', 'ADD', 'ADD']  impossible
@{} ['ADD', 'INV', 'ADD', 'INV']  impossible
@{} ['ADD', 'INV', 'INV', 'ADD']  impossible
@{} ['ADD', 'INV', 'INV', 'INV']  impossible
@{} ['INV', 'ADD', 'ADD', 'ADD']  impossible
@{} ['INV', 'ADD', 'ADD', 'INV']  impossible
@{} ['INV', 'ADD', 'INV', 'ADD']  impossible
@{} ['INV', 'ADD', 'INV', 'INV']

False

### How to decompose the projection function

From section 6.3, an expression of the form,

$ \mathrm{reduce}(\bigoplus, f_{p}, E)$

is equivalent to,

$ \mathrm{reduce}(\bigoplus, f_{p}'', \mathrm{reduce}(\bigoplus, f_{p}', E))$

where, $f_{p} = f_{p}'' \circ f_{p}'$ 

The projection function dictates which of the facets of the domain of the expression $E$ are considered strict boundaries. There are infinitely many choices for how to decompose the projection function here but we don't need to consider all of them.  We only to select decompositions that allow use to explore all possible combinations of boundary facets.

In other words, if we have four facets at a particular node in the face lattice then these are all of the ways that we could label them as boundaries (B) or non-boundaries (-):
```
c0,c1,c2,c3
[-,-,-,-]
[-,-,-,B]
[-,-,B,-]
[-,-,B,B]
[-,B,-,-]
[-,B,-,B]
[-,B,B,-]
[-,B,B,B]
[B,-,-,-]
[B,-,-,B]
[B,-,B,-]
[B,-,B,B]
[B,B,-,-]
[B,B,-,B]
[B,B,B,-]
[B,B,B,B]
```

For the current example, the constraints `C` are:

In [23]:
C

array([[  1,  -1,   0,   0],
       [  0,   0,   1,  -1],
       [  0,   1,   0,  -1],
       [ -1,   0,  -1, 100]])

In [24]:
# the root node of the face lattice
print(root)

frozenset()


In [25]:
# the current projection function
print(fp)

{[i,j,k]->[i]}


In [26]:
def show_which_facets_are_boundaries(fp):
    facets = list(lattice.graph.neighbors(root))
    for facet in facets:
        label = 'boundary' if is_strict(facet, root, fp, C) else ''
        print(set(facet), label)
    
# under the current projection function, none of the facets are strict boundaries
show_which_facets_are_boundaries(fp)

{0} 
{1} 
{2} 
{3} 


Different projection functions result in different facets being considered as strict boundaries:

In [27]:
show_which_facets_are_boundaries('{[i,j,k]->[i,i-j]}')

{0} boundary
{1} 
{2} boundary
{3} 


In [28]:
show_which_facets_are_boundaries('{[i,j,k]->[i-j,k]}')

{0} boundary
{1} boundary
{2} 
{3} 


To force a particular set of facets to "become" strict boundaries, choose an $f_{p}'$ such that:

$$ ker(f_{p}') \supseteq \big( ker(c_{0}) \cap ker(c_{1}) \cap \mathrm{...} \big) $$

In [29]:
labels = ['-', 'B']
facets = list(lattice.graph.neighbors(root))
for combo in enumerate_labels(facets, labels):
    print(combo)

['-', '-', '-', '-']
['-', '-', '-', 'B']
['-', '-', 'B', '-']
['-', '-', 'B', 'B']
['-', 'B', '-', '-']
['-', 'B', '-', 'B']
['-', 'B', 'B', '-']
['-', 'B', 'B', 'B']
['B', '-', '-', '-']
['B', '-', '-', 'B']
['B', '-', 'B', '-']
['B', '-', 'B', 'B']
['B', 'B', '-', '-']
['B', 'B', '-', 'B']
['B', 'B', 'B', '-']
['B', 'B', 'B', 'B']


For this case where we want $c_{1}$ and $c_{3}$ to be boundaries,  
```
['-', 'B', '-', 'B']
```

The null space of $c_{1}$ is:  
$\{[i,j,k] : k=0 \}$

The null space of $c_{3}$ is:  
$\{[i,j,k] : -k-i=0 \}$

And their intersection is:  
$I := \{[i,j,k] : k=0 \land i=0 \}$

There are two equalities in $I$, so the projection function that induces these labels must have two dimensions on the right hand side,
```
{[i,j,k]->[A,B]}
```

From here, we can construct the inner projection function $f_{p}'$ from $I$ (i.e., write the expresions A and B as functions of i, j, and k). This gives:  
```
{[i,j,k]->[k,i]}
```

In [30]:
show_which_facets_are_boundaries('{[i,j,k]->[k,i]}')

{0} 
{1} boundary
{2} 
{3} boundary


The final step is to then see if we can find $f_{p}''$ given this $f_{p}'$.  If an $f_{p}''$ can not be obtained, then conclude that this labeling is not possible and move to the next labeling. Else, the recursion proceeds by calling simplify with $f_{p}'$ instead. I haven't put this into my simplify code above yet, but here's what one of those calls would look like. Note that this can subsequently find a legal reuse vector to exploit (`[1,0,0]`):

In [31]:
simplify(k=1, fp_str='{[i,j,k]->[k,i]}', fd_str=fd, node=root, lattice=lattice, C=C, legal_labels=legal_labels)

@{} STEP A.1 - if k==0 return else continue. k=1
@{} 
@{} node = set()
@{} fp = {[i,j,k]->[k,i]}
@{} fd = {[i,j,k]->[j,k]}
@{} 
@{} STEP A.2 - identify "strict" boundaries given fp
@{} 
@{} {0} 
@{} {1} boundary
@{} {2} 
@{} {3} boundary
@{} 
@{} STEP A.3 - construct list of candidate facets (i.e., non-boundary facets)
@{} 
@{} candidate_facets = [{0}, {2}]
@{} 
@{} STEP A.4 - determine all possible combos
@{} 
@{} ['{0}', '{2}']
@{} --------------
@{} ['ADD', 'ADD']  impossible
@{} ['ADD', 'INV']  possible -> rho = { [1, 0, 0] }
@{} ['INV', 'ADD']  impossible
@{} ['INV', 'INV']  impossible
@{} 
@{} STEP A.5 - prune out redundant possible combos
@{} 
@{} ['{0}', '{2}']
@{} --------------
@{} ['ADD', 'INV']  -> rho = { [1, 0, 0] }
@{} 
@{} STEP A.6 - incorporate boundary facets
@{} 
@{} ['{0}', '{2}'] ['{1}', '{3}']
@{} -----------------------------
@{} ['ADD', 'INV'] ['inward', 'outward']  -> rho = { [1, 0, 0] }
@{} 
@{} STEP A.7 - recurse into "ADD" and "inward" boundary facets
@{} 
@

[['ADD', 'INV', 'inward', 'outward', Point("{ [1, 0, 0] }")]]